<a href="https://colab.research.google.com/github/Gouthammajjari/Intro_to_ml/blob/main/project/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Use the URL for the raw CSV data
url = 'https://raw.githubusercontent.com/Dhanush-adk/machine_learning/main/project/data_preprocessed.csv'
df = pd.read_csv(url)

# Display the first 5 rows of the DataFrame
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,day,hour,weekday,month,year,distance
0,7.5,-73.999817,40.738354,-73.999512,40.723217,1,7,19,3,5,2015,1.683323
1,7.7,-73.994355,40.728225,-73.994710,40.750325,1,17,20,4,7,2009,2.457590
2,12.9,-74.005043,40.740770,-73.962565,40.772647,1,24,21,0,8,2009,5.036377
3,5.3,-73.976124,40.790844,-73.965316,40.803349,3,26,8,4,6,2009,1.661683
4,16.0,-73.925023,40.744085,-73.973082,40.761247,5,28,17,3,8,2014,4.475450


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [4]:
X = df.drop('fare_amount', axis = 1)
y = df[['fare_amount']]

In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

In [7]:
X = pd.DataFrame(scaler_x.fit_transform(X), columns=X.columns, index=X.index).values
y = pd.DataFrame(scaler_y.fit_transform(y), columns=y.columns, index=y.index).values

In [8]:
train_X, val_X, train_Y, val_Y = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(train_X)
y_train =  torch.tensor(train_Y)
X_val =  torch.tensor(val_X)
y_val =  torch.tensor(val_Y)

In [9]:
training_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)

val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_data, batch_size=32)

In [10]:
def model_train(model, criterion, optimizer, epochs, train_loader, X_val, y_val):
    loss_train = []
    loss_val = []
    for epoch in range(epochs + 1):
        # Training
        model.train()
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.view(-1,1))
            loss.backward()
            optimizer.step()
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)
            if epoch % 50 == 0:
              print(f'Epoch {epoch}/{epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')
            loss_val.append(val_loss)
            loss_train.append(loss)
    return loss_train, loss_val

In [11]:
model1 = nn.Sequential(
    nn.Linear(X_train.shape[1], 32),
    nn.Tanh(),
    nn.Linear(32, 1)
).to(torch.float64)
optimizer1 = optim.SGD(model1.parameters(), lr=0.01)

In [ ]:
import time

start_time = time.time()
train_loss, val_loss = model_train(model1, nn.MSELoss(), optimizer1, 1000, train_loader, X_val, y_val)
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time: {training_time} seconds")

Epoch 0/1000, Training Loss: 0.0005382164196382274, Validation Loss: 0.000434840077555269
Epoch 50/1000, Training Loss: 3.4563302461946034e-05, Validation Loss: 0.00016004840857332378
Epoch 100/1000, Training Loss: 0.0001357767038681929, Validation Loss: 0.00015816808178789298
Epoch 150/1000, Training Loss: 4.8048005177145847e-05, Validation Loss: 0.00015704488962117898
Epoch 200/1000, Training Loss: 0.0002618534648789878, Validation Loss: 0.00015705976367616617
Epoch 250/1000, Training Loss: 7.083056782154282e-05, Validation Loss: 0.00015595119225042558
Epoch 300/1000, Training Loss: 7.771885469384481e-05, Validation Loss: 0.00015565565153723778
Epoch 350/1000, Training Loss: 3.427949858441738e-05, Validation Loss: 0.0001555375152286054
Epoch 400/1000, Training Loss: 2.344779575682929e-05, Validation Loss: 0.00015627342494261605
Epoch 450/1000, Training Loss: 4.043068301615955e-05, Validation Loss: 0.00015511204685005444
Epoch 500/1000, Training Loss: 3.083942723762982e-05, Validation

In [ ]:
torch.save(model1, 'model1.pth')

In [ ]:
import pickle

# Save list to a file
with open('train_loss1.pkl', 'wb') as file:
    pickle.dump(train_loss, file)
with open('val_loss1.pkl', 'wb') as file:
    pickle.dump(val_loss, file)

In [ ]:
model2 =  nn.Sequential(
    nn.Linear(X_train.shape[1], 32),
    nn.Tanh(),
    nn.Linear(32, 64),  # Second hidden layer with 64 neurons
    nn.Tanh(),
    nn.Linear(64, 16),  # Third hidden layer with 16 neurons
    nn.Tanh(),
    nn.Linear(16, 1) # Output layer with 1 neuron
).to(torch.float64)
criterion = nn.MSELoss()
optimizer1 = optim.SGD(model2.parameters(), lr=0.01)

In [ ]:
import time

start_time = time.time()
train_loss2, val_loss2 = model_train(model2, nn.MSELoss(), optimizer1, 1000, train_loader, X_val, y_val)
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time: {training_time} seconds")

In [ ]:
torch.save(model2, 'model2.pth')

In [ ]:
import pickle

# Save list to a file
with open('train_loss2.pkl', 'wb') as file:
    pickle.dump(train_loss2, file)
with open('val_loss2.pkl', 'wb') as file:
    pickle.dump(val_loss2, file)

# For more iterations

In [ ]:
model1 = nn.Sequential(
    nn.Linear(X_train.shape[1], 32),
    nn.Tanh(),
    nn.Linear(32, 1)
).to(torch.float64)
optimizer1 = optim.SGD(model1.parameters(), lr=0.01)

In [ ]:
import time

start_time = time.time()
train_loss_3000, val_loss_3000 = model_train(model1, nn.MSELoss(), optimizer1, 3000, train_loader, X_val, y_val)
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time: {training_time} seconds")

In [ ]:
torch.save(model1, 'model1_3000.pth')

In [ ]:
import pickle

# Save list to a file
with open('train_loss1_3000.pkl', 'wb') as file:
    pickle.dump(train_loss_3000 , file)
with open('val_loss1_3000.pkl', 'wb') as file:
    pickle.dump(val_loss_3000, file)

In [ ]:
model2 =  nn.Sequential(
    nn.Linear(X_train.shape[1], 32),
    nn.Tanh(),
    nn.Linear(32, 64),  # Second hidden layer with 64 neurons
    nn.Tanh(),
    nn.Linear(64, 16),  # Third hidden layer with 16 neurons
    nn.Tanh(),
    nn.Linear(16, 1) # Output layer with 1 neuron
).to(torch.float64)
criterion = nn.MSELoss()
optimizer1 = optim.SGD(model2.parameters(), lr=0.1)

In [ ]:
import time

start_time = time.time()
train_loss_2_3000, val_loss_2_3000 = model_train(model2, nn.MSELoss(), optimizer1, 3000, train_loader, X_val, y_val)
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time: {training_time} seconds")

In [ ]:

torch.save(model2, 'model2_3000.pth')

In [ ]:
import pickle

# Save list to a file
with open('train_loss_2_3000.pkl', 'wb') as file:
    pickle.dump(train_loss_2_3000, file)
with open('val_loss_2_3000.pkl', 'wb') as file:
    pickle.dump(val_loss_2_3000, file)

In [ ]:

print(model1)

In [ ]:
epochs = range(0, 1001, 50)

In [ ]:
def downsample_losses(train_loss, val_loss, step=50):
    train_loss_batch = [train_loss[i].detach().item() for i in range(0, len(train_loss), step)]
    val_loss_batch = [val_loss[i].detach().item() for i in range(0, len(val_loss), step)]
    return train_loss_batch, val_loss_batch

In [ ]:
train_loss_batch1, val_loss_batch1 = downsample_losses(train_loss, val_loss, step=50)
train_loss_batch2, val_loss_batch2 = downsample_losses(train_loss2, val_loss2, step=50)

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(2,2,1)
plt.plot(epochs, train_loss_batch1, color='blue', label = 'training Loss')
plt.plot(epochs, val_loss_batch1, color='red', label = 'validation Loss')
plt.title('Loss with Learning Rate of 0.01 (1 hidden layer)')
plt.legend()

plt.subplot(2,2,2)
plt.plot(epochs, train_loss_batch2, color='blue', label = 'training Loss')
plt.plot(epochs, val_loss_batch2, color='red', label = 'validation Loss')
plt.title('Loss with Learning Rate of 0.01 (3 Hidden Layers)')
plt.legend()

plt.show()

In [ ]:
train_loss_batch3, val_loss_batch3 = downsample_losses(train_loss_3000, val_loss_3000, step=50)
train_loss_batch4, val_loss_batch4 = downsample_losses(train_loss_2_3000, val_loss_2_3000, step=50)

In [ ]:
epochs = range(0, 3001, 50)

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(2,2,1)
plt.plot(epochs, train_loss_batch3, color='blue', label = 'training Loss')
plt.plot(epochs, val_loss_batch3, color='red', label = 'validation Loss')
plt.title('Loss with Learning Rate of 0.01 (1 hidden layer)')
plt.legend()

plt.subplot(2,2,2)
plt.plot(epochs, train_loss_batch4, color='blue', label = 'training Loss')
plt.plot(epochs, val_loss_batch4, color='red', label = 'validation Loss')
plt.title('Loss with Learning Rate of 0.01 (3 Hidden Layers)')
plt.legend()

plt.show()

In [ ]:
import torch

# Specify the path to your saved .pth file
model_path = 'model1.pth'

# Load the entire model (including architecture and parameters)
model = torch.load(model_path)

# Ensure the model is in evaluation mode (if it's a trained model)
model.eval()

# Print the trainable parameters
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable parameters ", total_params,  '\n')

In [ ]:
 import torch

# Specify the path to your saved .pth file
model_path = 'model2.pth'

# Load the entire model (including architecture and parameters)
model2 = torch.load(model_path)

# Ensure the model is in evaluation mode (if it's a trained model)
model2.eval()

# Print the trainable parameters
for name, param in model2.named_parameters():
    if param.requires_grad:
        print(name)

total_params = sum(p.numel() for p in model2.parameters() if p.requires_grad)
print("Total number of trainable parameters ", total_params,  '\n')

In [ ]:
import torch

# Specify the path to your saved .pth file
model_path = 'model1_3000.pth'

# Load the entire model (including architecture and parameters)
model1_3k = torch.load(model_path)

# Ensure the model is in evaluation mode (if it's a trained model)
model1_3k.eval()

# Print the trainable parameters
for name, param in model1_3k.named_parameters():
    if param.requires_grad:
        print(name)

total_params = sum(p.numel() for p in model1_3k.parameters() if p.requires_grad)
print("Total number of trainable parameters ", total_params,  '\n')

In [ ]:
import torch

# Specify the path to your saved .pth file
model_path = 'model2_3000.pth'

# Load the entire model (including architecture and parameters)
model2_3k = torch.load(model_path)

# Ensure the model is in evaluation mode (if it's a trained model)
model2_3k.eval()

# Print the trainable parameters
for name, param in model2_3k.named_parameters():
    if param.requires_grad:
        print(name)

total_params = sum(p.numel() for p in model2_3k.parameters() if p.requires_grad)
print("Total number of trainable parameters ", total_params,  '\n')